In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error,v_measure_score
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
import matplotlib as plt
%matplotlib inline
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cosine
from sklearn.manifold import TSNE

/home/anushya/proj1/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ratings_df = pd.read_csv('/home/anushya/Documents/ml-100k/u.data',sep="\t",header=None)
ratings_df.columns = ['user_id' ,'MovieID', 'rating','timestamp']
ratings_df

,user_id,MovieID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [3]:
user_count = ratings_df['user_id'].unique().shape[0]
items_count = ratings_df['MovieID'].unique().shape[0]
print (user_count,items_count)

943 1682


In [4]:
group_user = ratings_df.groupby('user_id')['rating'].count()
pd.DataFrame(group_user.sort_values(ascending = False))

,rating
user_id,
405,737
655,685
13,636
450,540
276,518
416,493
537,490
303,484
234,480







# Split the data using crossvalidation

In [5]:
 train_data, test_data = cv.train_test_split(ratings_df, test_size = 0.25)




# Creating Matrix from Dataframe

In [6]:
train_matrix = np.zeros((user_count, items_count))
for line in train_data.itertuples():
    train_matrix[line[1]-1, line[2]-1] = line[3]
test_matrix = np.zeros((user_count, items_count))
for line in test_data.itertuples():
    test_matrix[line[1]-1, line[2]-1] = line[3]
data_df = pd.DataFrame(train_matrix)
data_df.reset_index()



,index,0,1,2,3,4,5,6,7,8,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0,5.0,3.0,0.0,3.0,3.0,0.0,4.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
user_similarity = 1-(pairwise_distances(train_matrix, metric='cosine'))
item_similarity = 1-(pairwise_distances(train_matrix.T, metric='cosine'))
df = pd.DataFrame(user_similarity)
#df[3]

In [8]:
k = df[3].sort_values(ascending = False).head(10)
k

3      1.000000
800    0.371239
301    0.369274
749    0.351223
508    0.338093
172    0.333926
790    0.333799
383    0.329864
2      0.321109
826    0.317604
Name: 3, dtype: float64

In [9]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred


In [10]:
item_prediction = predict(train_matrix, item_similarity, type='item')
user_prediction = predict(train_matrix, user_similarity, type='user')
df1 = pd.DataFrame(user_prediction)
df1
#user_prediction

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.033186,0.740050,0.495929,1.211291,0.542648,0.280711,1.842196,1.202411,1.442494,0.597663,...,0.202636,0.199201,0.199201,0.199201,0.199201,0.199432,0.199894,0.199663,0.199201,0.206465
1,1.425162,0.170674,0.154608,0.405973,0.131030,0.026255,1.080463,0.491306,1.017496,0.285864,...,-0.050409,-0.053111,-0.053111,-0.053111,-0.053111,-0.051710,-0.048908,-0.050309,-0.053111,-0.051529
2,0.981474,0.140324,0.100864,0.305376,0.090074,-0.030238,0.838853,0.354790,0.641588,0.144006,...,-0.075358,-0.076813,-0.076813,-0.076813,-0.076813,-0.072912,-0.065112,-0.069012,-0.076813,-0.075316
3,1.119259,0.158690,0.092421,0.334147,0.081153,-0.044398,0.909691,0.384303,0.640688,0.112438,...,-0.090001,-0.092859,-0.092859,-0.092859,-0.092859,-0.089889,-0.083947,-0.086918,-0.092859,-0.090920
4,1.854825,0.573863,0.283497,1.060953,0.344474,0.052307,1.621697,1.040883,1.083400,0.359085,...,-0.011116,-0.014505,-0.014505,-0.014505,-0.014505,-0.014498,-0.014483,-0.014490,-0.014505,-0.008463
5,1.752649,0.553029,0.351861,1.003807,0.375175,0.198579,1.588231,1.116799,1.357897,0.514524,...,0.111763,0.106776,0.106776,0.106776,0.106776,0.106938,0.107264,0.107101,0.106776,0.112558
6,2.163559,1.010858,0.725920,1.465899,0.791770,0.542359,2.027139,1.496228,1.632404,0.882025,...,0.463743,0.458715,0.458715,0.458715,0.458715,0.458950,0.459419,0.459184,0.458715,0.464943
7,1.608021,0.485568,0.145910,0.871025,0.187310,-0.066637,1.508220,0.797906,0.932474,0.207707,...,-0.125797,-0.127912,-0.127912,-0.127912,-0.127912,-0.127841,-0.127698,-0.127769,-0.127912,-0.122272
8,1.405664,0.244717,0.117873,0.569834,0.102229,0.009451,1.496581,0.643929,0.959328,0.190319,...,-0.140591,-0.143087,-0.143087,-0.143087,-0.143087,-0.142088,-0.140091,-0.141090,-0.143087,-0.140084
9,1.711269,0.571201,0.363055,1.042133,0.395290,0.211325,1.639704,1.117819,1.331307,0.529020,...,0.114798,0.109395,0.109395,0.109395,0.109395,0.109593,0.109989,0.109791,0.109395,0.115416


In [11]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [12]:
rmse(item_prediction, test_matrix)

3.1657435748108136

In [13]:
rmse(user_prediction, test_matrix)

2.9587580215704214

In [14]:


knn = KNeighborsRegressor(n_neighbors=5)

# Recommendation

In [15]:
def predict_topk(ratings, similarity, kind='user', k=20):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
    if kind == 'item':
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items]))        
    
    return pred

In [16]:
pred = predict_topk(train_matrix, user_similarity, kind='user', k=20)
pred1 = predict_topk(train_matrix, item_similarity, kind='item', k=20)




In [17]:
pred

array([[ 3.29467023,  1.41706888,  1.23547922, ...,  0.        ,
         0.        ,  0.146769  ],
       [ 2.48265369,  0.        ,  0.26775696, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.09381119,
         0.        ,  0.        ],
       ..., 
       [ 3.23142811,  0.        ,  0.39044885, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.82622992,  0.17759399,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.27911247,  2.05681228,  0.7229494 , ...,  0.        ,
         0.        ,  0.        ]])

In [18]:
pred1

array([[ 3.34783139,  1.89245255,  0.98890625, ...,  0.        ,
         0.        ,  0.20857126],
       [ 0.98086293,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.21484093,  0.        ,  0.        , ...,  0.07160246,
         0.        ,  0.        ],
       ..., 
       [ 0.86901516,  0.        ,  0.41807034, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.69347869,  0.21606481,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.49356386,  2.1667042 ,  1.4942414 , ...,  0.        ,
         0.        ,  0.        ]])

creating a table of user_id,item_id

In [19]:
R_df = ratings_df.pivot(index ='user_id' , columns = 'MovieID',values = 'rating').fillna(0)
R_df.head()


MovieID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## converting df to input matrix for svd

In [20]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demean = R - user_ratings_mean.reshape(-1, 1)
R_demean

array([[ 4.41617122,  2.41617122,  3.41617122, ..., -0.58382878,
        -0.58382878, -0.58382878],
       [ 3.86325803, -0.13674197, -0.13674197, ..., -0.13674197,
        -0.13674197, -0.13674197],
       [-0.08977408, -0.08977408, -0.08977408, ..., -0.08977408,
        -0.08977408, -0.08977408],
       ..., 
       [ 4.9470868 , -0.0529132 , -0.0529132 , ..., -0.0529132 ,
        -0.0529132 , -0.0529132 ],
       [-0.20035672, -0.20035672, -0.20035672, ..., -0.20035672,
        -0.20035672, -0.20035672],
       [-0.34066587,  4.65933413, -0.34066587, ..., -0.34066587,
        -0.34066587, -0.34066587]])

svd

In [21]:
U, sigma, Vt = svds(R_demean, k = 50)
sigma = np.diag(sigma)


prediction

In [22]:
user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
pred_df = pd.DataFrame(user_predicted_ratings,columns = R_df.columns)
pred_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,...,0.011976,-0.092017,-0.074553,-0.060985,0.009427,-0.035641,-0.039227,-0.037434,-0.025552,0.023513
1,2.347262,0.129689,-0.098917,0.328828,0.159517,0.481361,0.213002,0.097908,1.892100,0.671000,...,0.003943,-0.026939,-0.035460,-0.029883,-0.027153,-0.015244,-0.008277,-0.011760,0.011639,-0.046924
2,0.291905,-0.263830,-0.151454,-0.179289,0.013462,-0.088309,-0.057624,0.568764,-0.018506,0.280742,...,-0.028964,-0.031622,0.045513,0.026089,-0.021705,0.002282,0.032363,0.017322,-0.006644,-0.009480
3,0.366410,-0.443535,0.041151,-0.007616,0.055373,-0.080352,0.299015,-0.010882,-0.160888,-0.118834,...,0.020069,0.015981,-0.000182,0.005593,0.026634,0.023562,0.036405,0.029984,0.015612,-0.008713
4,4.263488,1.937122,0.052529,1.049350,0.652765,0.002836,1.730461,0.870584,0.341027,0.569055,...,0.019973,-0.053521,-0.017242,-0.007137,-0.038987,0.010338,0.004869,0.007603,-0.020575,0.003330


In [23]:
item_data = pd.read_csv('/home/anushya/Documents/ml-100k/u.item',encoding='latin-1',sep = '|',header=None)
item_data.columns = ['movie id','movie title', 'release date' ,'video release date' ,
              'IMDb URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Children' , 'Comedy' ,'Crime' , 'Documentary ', 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' ,'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' ,' Western']
#item_data.head()

In [24]:
movies_list = [i.strip().split("::") for i in open('/home/anushya/Documents/ml-100k/movies.dat', 'r', encoding='ISO-8859-1').readlines()]
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)
movies_df

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [48]:


def recommend_movies(predictions_df, userID, movies_df, ratings_df, num_recommendations=5):
    user_row_number = userID - 1
    sorted_user_predictions = pred_df.iloc[user_row_number].sort_values(ascending=False) 
    user_data = ratings_df[ratings_df.user_id == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').sort_values(['rating'], ascending=False))

   # print (format(userID, user_full.shape[0]
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',left_on = 'MovieID',right_on = 'MovieID')
.rename(columns = {user_row_number: 'Predictions'})
                       .sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    return user_full, recommendations



In [49]:
sorted_user_predictions = pred_df.iloc[500].sort_values(ascending=False) 
#sorted_user_predictions
#pd.DataFrame(sorted_user_predictions).reset_index()

In [53]:
already_rated, predictions = recommend_movies(pred_df,4, movies_df, ratings_df, 10)

In [54]:
already_rated

,user_id,MovieID,rating,timestamp,Title,Genres
23,4,301,5,892002353,Picture Bride (1995),Drama|Romance
19,4,359,5,892002352,I Like It Like That (1994),Comedy|Drama|Romance
17,4,327,5,892002352,Tank Girl (1995),Action|Comedy|Musical|Sci-Fi
15,4,329,5,892002352,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
20,4,362,5,892002352,"Jungle Book, The (1994)",Adventure|Children's|Romance
13,4,258,5,892001374,"Kid in King Arthur's Court, A (1995)",Adventure|Children's|Comedy|Fantasy|Romance
1,4,303,5,892002352,"Quick and the Dead, The (1995)",Action|Adventure|Western
11,4,300,5,892001445,Quiz Show (1994),Drama
9,4,354,5,892002353,Cobb (1994),Drama
8,4,50,5,892003526,"Usual Suspects, The (1995)",Crime|Thriller


In [55]:
predictions

,MovieID,Title,Genres
177,181,Mighty Morphin Power Rangers: The Movie (1995),Action|Children's
714,748,"Arrival, The (1996)",Action|Sci-Fi|Thriller
288,302,Queen Margot (La Reine Margot) (1994),Drama|Romance
259,268,Little Odessa (1994),Drama
313,333,Tommy Boy (1995),Comedy
260,269,My Crazy Life (Mi vida loca) (1993),Drama
309,326,To Live (Huozhe) (1994),Drama
312,332,Village of the Damned (1995),Horror|Sci-Fi
320,340,"War, The (1994)",Adventure|Drama
278,289,Only You (1994),Comedy|Romance
